In [12]:
import pandas as pd

In [13]:
df =pd.read_csv('../data/data.csv',usecols=lambda x: x != "id")

In [14]:
!wget https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py

--2025-08-05 21:15:52--  https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘minsearch.py’

minsearch.py            [ <=>                ] 190.14K  --.-KB/s    in 0.002s  

2025-08-05 21:15:52 (75.8 MB/s) - ‘minsearch.py’ saved [194699]



In [15]:
df.columns

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [16]:
!more minserach.py

more: cannot open minserach.py: No such file or directory


In [17]:
ls

RAG_test.ipynb  minsearch.py
